## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/ContainsNull.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

Id,Name,Sales
emp1,John,null
emp2,null,null
emp3,null,345.0
emp4,Cindy,456.0


In [3]:
# Create a view or table

temp_table_name = "ContainsNull_csv"

df.createOrReplaceTempView(temp_table_name)

In [4]:
%sql

/* Query the created temp table in a SQL cell */

select * from `ContainsNull_csv`

Id,Name,Sales
emp1,John,null
emp2,null,null
emp3,null,345.0
emp4,Cindy,456.0


In [5]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "ContainsNull_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [6]:
df.na.drop(thresh=2).show() # drops rows with atleast 2 null values in it.

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [7]:
df.na.drop(how = 'all').show() # drops rows with all the columns as null

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [8]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [9]:
df.printSchema()

root
-- Id: string (nullable = true)
-- Name: string (nullable = true)
-- Sales: double (nullable = true)

In [10]:
df.na.fill('No Name').show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|No Name| null|
emp3|No Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [11]:
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|No Name| null|
emp3|No Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [12]:
from pyspark.sql.functions import mean

In [13]:
mean_val = df.select(mean(df['Sales'])).collect()

In [14]:
mean_sales = mean_val[0][0]

In [15]:
df.na.fill(mean_sales,['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+